In [4]:
#We will be using a Deep Learning Model because we have 5 Classifications (1-5 star yelp ratings) and whether those correlate to a 1-3 Michelin Star resturant.

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
michelin_star_df = pd.read_csv("michelin_data.csv") 
michelin_star_df.head()

,name,year,latitude,longitude,city,region,zipCode,cuisine,price,url,stars
0,Kilian Stuba,2019,47.348580,10.17114,Kleinwalsertal,Austria,87568,Creative,$$$$$,https://guide.michelin.com/at/en/vorarlberg/kl...,1
1,Pfefferschiff,2019,47.837870,13.07917,Hallwang,Austria,5300,Classic cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...,1
2,Esszimmer,2019,47.806850,13.03409,Salzburg,Austria,5020,Creative,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...,1
3,Carpe Diem,2019,47.800010,13.04006,Salzburg,Austria,5020,Market cuisine,$$$$$,https://guide.michelin.com/at/en/salzburg-regi...,1
4,Edvard,2019,48.216503,16.36852,Wien,Austria,1010,Modern cuisine,$$$$,https://guide.michelin.com/at/en/vienna/wien/r...,1


In [6]:
# Drop any non-beneficial columns.
michelin_star_df = michelin_star_df.drop(columns =['url', 'price'])
michelin_star_df

,name,year,latitude,longitude,city,region,zipCode,cuisine,stars
0,Kilian Stuba,2019,47.348580,10.171140,Kleinwalsertal,Austria,87568,Creative,1
1,Pfefferschiff,2019,47.837870,13.079170,Hallwang,Austria,5300,Classic cuisine,1
2,Esszimmer,2019,47.806850,13.034090,Salzburg,Austria,5020,Creative,1
3,Carpe Diem,2019,47.800010,13.040060,Salzburg,Austria,5020,Market cuisine,1
4,Edvard,2019,48.216503,16.368520,Wien,Austria,1010,Modern cuisine,1
...,...,...,...,...,...,...,...,...,...
690,Fat Duck,2019,51.508280,-0.702320,Bray,United Kingdom,SL6 2AQ,Creative,3
691,Waterside Inn,2019,51.507730,-0.701210,Bray,United Kingdom,SL6 2AT,Classic French,3
692,Alain Ducasse at The Dorchester,2019,51.507120,-0.152520,Mayfair,United Kingdom,W1K 1QA,French,3
693,The Araki,2019,51.511826,-0.140389,Mayfair,United Kingdom,W1S 3BF,Japanese,3


In [ ]:
# Determine the number of unique values in each column.
michelin_star_df.nunique()

In [ ]:
# Look at Yelp Star Review value counts for binning
yelp_counts = michelin_star_df.YELPSTARREVIEWS.value_counts()
yelp_counts

In [ ]:
# Visualize the value counts of YELPSTARREVIEWS
yelp_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...? May need to write a for loop in the case that the numbers are between whole numbers (4.6,etc.)
replace_yelp = list(yelp_counts[yelp_counts < 528].index)

# Replace in dataframe
for app in replace_yelp:
    michelin_star_df.YELPSTARREVIEWS = michelin_star_df.YELPSTARREVIEWS.replace(app,"Other")
    
# Check to make sure binning was successful
michelin_star_df.YELPSTARREVIEWS.value_counts()

In [ ]:
# Look at MICHELINSTARS value counts for binning
michelinstars_counts = michelin_star_df.MICHELINSTARS.value_counts()
michelinstars_counts

In [ ]:
# Visualize the value counts of MICHELINSTARS
michelinstars_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ..? # Same as before. THis may not be necessary. 
replace_michelin = list(michelinstars_counts[michelinstars_counts < 1883].index)

# Replace in dataframe
for cls in replace_michelin:
   michelin_star_df.MICHELINSTARS = michelin_star_df.MICHELINSTARS.replace(cls,"Other")
    
# Check to make sure binning was successful
michelin_star_df.MICHELINSTARS.value_counts()

In [9]:
# Generate our categorical variable lists
michelin_cat = michelin_star_df.dtypes[michelin_star_df.dtypes == "object"].index.tolist()

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(michelin_star_df[michelin_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(michelin_cat)
encode_df.head()

/Users/erinoneil/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,name_108,name_21212,name_28+,name_360º,name_8 1/2 Otto e Mezzo - Bombana,name_8½ Otto e Mezzo - Bombana,name_A. Wong,name_Acadia,name_Acquerello,name_Adam's,...,cuisine_Taiwanese,cuisine_Taizhou,cuisine_Temple cuisine,cuisine_Teppanyaki,cuisine_Thai,cuisine_Thai Contemporary,cuisine_Traditional British,cuisine_Vegetarian,cuisine_creative,cuisine_modern
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
michelin_star_df = michelin_star_df.merge(encode_df,left_index=True, right_index=True)
michelin_star_df = michelin_star_df.drop(michelin_cat,1)
michelin_star_df.head()

/Users/erinoneil/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,year,latitude,longitude,stars,name_108,name_21212,name_28+,name_360º,name_8 1/2 Otto e Mezzo - Bombana,name_8½ Otto e Mezzo - Bombana,...,cuisine_Taiwanese,cuisine_Taizhou,cuisine_Temple cuisine,cuisine_Teppanyaki,cuisine_Thai,cuisine_Thai Contemporary,cuisine_Traditional British,cuisine_Vegetarian,cuisine_creative,cuisine_modern
0,2019,47.348580,10.17114,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019,47.837870,13.07917,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019,47.806850,13.03409,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019,47.800010,13.04006,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019,48.216503,16.36852,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Split our preprocessed data into our features and target arrays
y = michelin_star_df(['latitude', 'stars'])
X = michelin_star_df.drop(['year', 'stars'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

TypeError: 'DataFrame' object is not callable

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}hdf5"

In [ ]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")